# 损失函数

"SMAPE loss" 是指 Symmetric Mean Absolute Percentage Error，中文意为对称平均绝对百分比误差。这是一种用于衡量预测模型精度的指标，通常用于评估时间序列预测模型的性能。

SMAPE 的计算公式如下：

\$
SMAPE = \frac{100\%}{n} \sum_{t=1}^{n} \frac{|F_t - A_t|}{(|F_t| + |A_t|)/2}
\$

其中：

- \(F_t\) 是模型预测的值；
- \(A_t\) 是实际观测到的值；
- \(n\) 是样本数量。

SMAPE 通常被用作损失函数，用于优化时间序列预测模型。在优化过程中，目标是尽量减小 SMAPE 损失，以提高模型的预测准确性。

In [1]:
# 损失函数
class smape_loss(nn.Module):
    def __init__(self):
        super(smape_loss, self).__init__()

    def divide_no_nan(self, a, b):
        """
        a/b where the resulted NaN or Inf are replaced by 0.
        """
        result = a / b
        result[result != result] = .0
        result[result == np.inf] = .0
        return result
    
    def sequence_mask(self, target: torch.Tensor):
        np.random.seed(2024)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        batch_size, pred_len = target.shape[0], target.shape[1]
        mask = torch.triu(torch.ones((pred_len, 1), dtype=torch.uint8),
                        diagonal=1)
        cut_point = np.random.randint(low=1, high=pred_len, size=1)[0]
        mask[:cut_point, 0] = 1.0
        mask.unsqueeze(0).expand(batch_size, -1, -1)  # [B, L, L]
        return mask.to(device)

    def forward(self, forecast: torch.Tensor, target: torch.Tensor) -> torch.float:
        """
        sMAPE loss as defined in https://robjhyndman.com/hyndsight/smape/ (Makridakis 1993)

        :param forecast: Forecast values. Shape: batch, time
        :param target: Target values. Shape: batch, time
        :param mask: 0/1 mask. Shape: batch, time
        :return: Loss value
        """
        mask = self.sequence_mask(target)
        return 200 * torch.mean(self.divide_no_nan(torch.abs(forecast - target),
                                          torch.abs(forecast.data) + torch.abs(target.data))* mask)

NameError: name 'nn' is not defined

In [ ]:
def sequence_mask(target: torch.Tensor):
    np.random.seed(2024)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size, seq_len = target.shape[0], target.shape[1]
    mask = torch.triu(torch.ones((seq_len, 2), dtype=torch.uint8),
                    diagonal=1)
    cut_point = np.random.randint(low=1, high=seq_len, size=1)[0]
    mask[-cut_point:, 0] = 1.0
    mask = mask.unsqueeze(0).expand(batch_size, -1, -1)  # [B, L, L]
    return mask.to(device)